# 3D FEA — scikit-fem (Full Spring, Extruded)

Full 3D finite element analysis of the **complete doubly-clamped bistable spring**
using `scikit-fem` with tetrahedral elements.

The merged 2D polygon is extruded by the structural thickness (0.5 µm) in z
using `gmsh`. No plane strain assumption — full 3D elasticity.

**Boundary conditions**:
- Left anchor (x≈0): clamped, u_x = u_y = u_z = 0
- Right anchor (x≈anchor_distance): clamped, u_x = u_y = u_z = 0
- Shuttle (center): prescribe u_y, u_x and u_z free

Runs at 3 mesh densities (10%, 50%, 100%) for convergence study.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

import gmsh
import meshio
import skfem
from skfem import *
from skfem.models.elasticity import linear_elasticity, lame_parameters
from skfem.helpers import ddot, transpose, grad
from skfem.assembly import BilinearForm, LinearForm

sys.path.insert(0, os.path.dirname(os.path.abspath("__file__")))
from full_spring_utils import (
    POLY_SI, DEFAULT_FULL_SPRING_PARAMS,
    get_full_spring_polygon, get_full_spring_3d_mesh,
    identify_bc_nodes_3d,
)

print(f"scikit-fem version: {skfem.__version__}")

P = DEFAULT_FULL_SPRING_PARAMS
anchor_dist = P['anchor_distance']
half_span = P['half_span']
shuttle_len = P['shuttle_length']
h = P['initial_offset']
t = POLY_SI['t']
E = POLY_SI['E']
nu = POLY_SI['nu']
lam, mu = lame_parameters(E, nu)

print(f"Full spring: anchor_dist={anchor_dist} µm, half_span={half_span} µm")
print(f"  h={h} µm, t={t} µm, Q={h/t:.2f}")
print(f"E={E/1e3:.0f} GPa, ν={nu}, λ={lam:.1f} MPa, μ={mu:.1f} MPa")

os.makedirs('plots', exist_ok=True)
os.makedirs('results', exist_ok=True)

## 1. Nonlinear Forms (3D Total Lagrangian)

In [ ]:
I3 = np.eye(3).reshape(3, 3, 1, 1)

def mat(A, B):
    """Matrix-matrix product for (3,3,...) tensor fields."""
    return np.einsum('ij...,jk...->ik...', A, B)

@BilinearForm
def stiffness_nl_3d(u, v, w):
    """3D tangent stiffness: Total Lagrangian, St. Venant-Kirchhoff."""
    du = grad(w['prev'])
    F = I3 + du
    GL = 0.5 * (mat(transpose(F), F) - I3)
    trE = GL[0,0] + GL[1,1] + GL[2,2]
    S = lam * trE * I3 + 2.0 * mu * GL
    
    dv = grad(v)
    dw = grad(u)
    
    dGL = 0.5 * (mat(transpose(F), dw) + mat(transpose(dw), F))
    trDE = dGL[0,0] + dGL[1,1] + dGL[2,2]
    dS = lam * trDE * I3 + 2.0 * mu * dGL
    dGL_v = 0.5 * (mat(transpose(F), dv) + mat(transpose(dv), F))
    
    k_mat = ddot(dS, dGL_v)
    k_geo = ddot(S, 0.5 * (mat(transpose(dw), dv) + mat(transpose(dv), dw)))
    return k_mat + k_geo

@LinearForm
def internal_forces_3d(v, w):
    """3D internal force vector."""
    du = grad(w['prev'])
    F = I3 + du
    GL = 0.5 * (mat(transpose(F), F) - I3)
    trE = GL[0,0] + GL[1,1] + GL[2,2]
    S = lam * trE * I3 + 2.0 * mu * GL
    dv = grad(v)
    dGL_v = 0.5 * (mat(transpose(F), dv) + mat(transpose(dv), F))
    return ddot(S, dGL_v)

print("3D nonlinear forms defined.")

## 2. Mesh Density Sweep

Three density levels with varying mesh characteristic length and z-layers.

In [ ]:
densities = {
    '10pct': {'lc_flex': 0.8, 'lc_rigid': 1.2, 'n_layers_z': 2,
              'n_nl_steps': 20, 'label': '10% density'},
    '50pct': {'lc_flex': 0.5, 'lc_rigid': 0.8, 'n_layers_z': 2,
              'n_nl_steps': 25, 'label': '50% density'},
    '100pct': {'lc_flex': 0.3, 'lc_rigid': 0.5, 'n_layers_z': 3,
               'n_nl_steps': 30, 'label': '100% density'},
}

all_results = {}

# Generate polygon once
poly = get_full_spring_polygon(n_points=300)

In [ ]:
max_newton = 40
nl_tol = 1e-6

for density_name, cfg in densities.items():
    print(f"\n{'='*60}")
    print(f"Density: {cfg['label']}")
    print(f"  lc_flex={cfg['lc_flex']}, lc_rigid={cfg['lc_rigid']}, n_z={cfg['n_layers_z']}")
    print(f"{'='*60}")
    
    # --- 3D mesh via gmsh ---
    points_3d, tets = get_full_spring_3d_mesh(
        polygon=poly,
        thickness=t,
        lc_flex=cfg['lc_flex'],
        lc_rigid=cfg['lc_rigid'],
        n_layers_z=cfg['n_layers_z'],
        anchor_distance=anchor_dist,
        half_span=half_span,
        flex_ratio=P['flex_ratio'],
    )
    print(f"Mesh: {len(points_3d)} nodes, {len(tets)} tetrahedra")
    
    # --- scikit-fem mesh ---
    mesh3d = MeshTet(
        np.ascontiguousarray(points_3d.T, dtype=np.float64),
        np.ascontiguousarray(tets.T, dtype=np.int64),
    )
    
    # --- Basis ---
    ib3d = Basis(mesh3d, ElementVector(ElementTetP1()))
    N3d = mesh3d.nvertices
    ndofs_3d = 3 * N3d
    print(f"System DOFs: {ndofs_3d}")
    
    # --- Boundary nodes ---
    bc = identify_bc_nodes_3d(points_3d, anchor_dist, half_span, shuttle_len)
    print(f"Left anchor: {len(bc['left_anchor'])} nodes")
    print(f"Right anchor: {len(bc['right_anchor'])} nodes")
    print(f"Shuttle: {len(bc['shuttle'])} nodes")
    
    # DOFs: interleaved [x0,y0,z0, x1,y1,z1, ...]
    left_x = 3 * bc['left_anchor']
    left_y = 3 * bc['left_anchor'] + 1
    left_z = 3 * bc['left_anchor'] + 2
    right_x = 3 * bc['right_anchor']
    right_y = 3 * bc['right_anchor'] + 1
    right_z = 3 * bc['right_anchor'] + 2
    fixed_dofs = np.concatenate([left_x, left_y, left_z, right_x, right_y, right_z])
    
    shuttle_y_dofs = 3 * bc['shuttle'] + 1
    all_bc_3d = np.concatenate([fixed_dofs, shuttle_y_dofs])
    
    # --- Nonlinear sweep ---
    n_steps = cfg['n_nl_steps']
    delta_vals = np.linspace(0, -2*h, n_steps)
    F_nl = np.zeros(n_steps)
    
    u_curr = np.zeros(ndofs_3d)
    
    for i, delta in enumerate(delta_vals):
        u_iter = u_curr.copy()
        u_iter[fixed_dofs] = 0.0
        u_iter[shuttle_y_dofs] = delta
        
        converged = False
        for nit in range(max_newton):
            u_field = ib3d.interpolate(u_iter)
            Kt = stiffness_nl_3d.assemble(ib3d, prev=u_field)
            fint = internal_forces_3d.assemble(ib3d, prev=u_field)
            
            residual = fint.copy()
            residual[all_bc_3d] = 0.0
            rn = np.linalg.norm(residual)
            
            if rn < nl_tol:
                converged = True
                break
            
            du_c = solve(*condense(Kt, -residual, D=all_bc_3d))
            u_iter += du_c
        
        if not converged and i > 0:
            print(f"  Step {i}: δ={delta:.4f}, not converged (res={rn:.2e})")
        
        u_curr = u_iter.copy()
        
        u_field = ib3d.interpolate(u_curr)
        fint = internal_forces_3d.assemble(ib3d, prev=u_field)
        F_nl[i] = np.sum(fint[shuttle_y_dofs])
        
        if i % 10 == 0:
            print(f"  Step {i}/{n_steps}: δ={delta:.3f} µm, F={F_nl[i]:.4f} µN, "
                  f"Newton iters={nit+1}")
    
    print(f"Done. F range: [{F_nl.min():.4f}, {F_nl.max():.4f}] µN")
    
    all_results[density_name] = {
        'delta': -delta_vals,
        'F': F_nl,
        'n_nodes': len(points_3d),
        'n_elems': len(tets),
    }
    
    np.savetxt(f'results/skfem_3d_full_{density_name}.csv',
               np.column_stack([-delta_vals, F_nl]),
               delimiter=',', header='delta_um,F_uN', comments='')
    print(f"Saved: results/skfem_3d_full_{density_name}.csv")

## 3. Mesh Convergence Plot

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors = {'10pct': 'green', '50pct': 'blue', '100pct': 'red'}
for name, res in all_results.items():
    label = f"{densities[name]['label']} ({res['n_nodes']} nodes, {res['n_elems']} tets)"
    ax.plot(res['delta'], res['F'], color=colors[name], linewidth=1.5, label=label)

try:
    ana = np.loadtxt('results/analytical_force_displacement.csv',
                     delimiter=',', skiprows=1)
    ax.plot(ana[:, 0], 4 * ana[:, 2], 'k:', linewidth=1.5,
            label='Analytical CCS (×4)')
except FileNotFoundError:
    pass

ax.axhline(0, color='gray', linewidth=0.5)
ax.set_xlabel('Shuttle displacement from initial [µm]')
ax.set_ylabel('Force F [µN]')
ax.set_title('scikit-fem 3D Full Spring — Mesh Convergence')
ax.legend()
ax.grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig('plots/skfem_3d_full_convergence.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Force-Displacement Summary

In [ ]:
print(f"{'Density':15s} {'Nodes':>8s} {'Tets':>8s} {'F_max [µN]':>12s} {'F_min [µN]':>12s}")
print('-' * 60)
for name, res in all_results.items():
    print(f"{densities[name]['label']:15s} {res['n_nodes']:8d} {res['n_elems']:8d} "
          f"{res['F'].max():12.4f} {res['F'].min():12.4f}")

print(f"\nExpected: F_push ≈ 10 µN (4 × 2.48 analytical CCS half-beam)")
print(f"Expected: snap-through at δ ≈ {h:.1f} µm")